# Atividade 3

Faça tudo via dbt. 

Envia como resposta:
- A query SQL
- Comando utilizado via terminal no `dbt`
- Um print do resultado no próprio notebook, para isso é necessário criar uma pasta no repositório onde serão armazenadas as imagens, e para inserir no notebook é necessário criar uma célula `markdown` e inserir o seguinte comando: 

```markdown
![Nome da Foto](path/para/imagem)
```

1. Crie uma tabela `orders_by_year` a partir de `orders` que agrupa a quantidade de pedidos por ano, considerando a coluna `order_approved_at`

A query SQL

orders_by_year.sql
   
   ```sql
   {{ config(materialized='table') }}

    SELECT
        FORMAT_DATE('%Y', order_approved_at) AS year, 
        count(order_id) AS num_orders 
    FROM 
        `dbt_shop.orders` 
    GROUP BY 
        1
```

Comando utilizado via terminal no dbt

``dbt run --models shop.orders_by_year``

Prints

![Comando no Terminal](atividade3/image-1.png)
![Comando no Terminal](atividade3/image-2.png)
![Comando no Terminal](atividade3/image-3.png)


2. Crie uma `view` que agrega os totais de pedidos por estado. Considere as tabelas _customers_ para obter os estados (coluna _customer_state_) e _orders_ para contabilizar a quantidade de pedidos.

A query SQL

pedidos_por_estado.sql
   
   ```sql
{{ config(materialized='view') }}

    SELECT
        customers.customer_state AS state, 
        count(order_id) AS num_orders 
    FROM 
        `dbt_shop.customers` AS customers
    INNER JOIN `dbt_shop.orders` AS orders
    ON customers.customer_id = orders.customer_id
    GROUP BY 1
    ORDER BY 1
```

Comando utilizado via terminal no dbt

``dbt run --models shop.pedidos_por_estado``

Prints

![Comando no Terminal](atividade3/image-4.png)
![Comando no Terminal](atividade3/image-5.png)
![Comando no Terminal](atividade3/image-6.png)

3. De acordo com o resultado do exercício 2 retorne apenas os estados que tem mais de 5000 pedidos, esse número deve ser passado via variável `number_of_orders` do `dbt`.

A query SQL

pedidos_por_estado_variavel.sql
   
   ```sql
{{ config(materialized='view') }}

    SELECT
        customers.customer_state AS state, 
        count(order_id) AS num_orders 
    FROM 
        `dbt_shop.customers` AS customers
    INNER JOIN `dbt_shop.orders` AS orders
    ON customers.customer_id = orders.customer_id
    GROUP BY 1
    HAVING num_orders > {{ var("number_of_orders") }}
    ORDER BY 1
```

Comando utilizado via terminal no dbt

``dbt run --models shop.pedidos_por_estado_variavel --vars "{'number_of_orders': '5000'}"``

Prints

![Comando no Terminal](atividade3/image-7.png)
![Comando no Terminal](atividade3/image-8.png)
![Comando no Terminal](atividade3/image-9.png)


4. Crie uma `view` chamada `customers_by_region` que utiliza os dados da tabela `customers`. A visão deve realizar uma agregação por região do Brasil (Norte, Nordeste, Sudeste, Centro Oeste e Sul) e contar o número de clientes em cada uma delas.

A query SQL

customers_by_region.sql
   
   ```sql
{{ config(materialized='view') }}

    SELECT region.region as region,
        sum(region.customers) as customers
    FROM 
        (SELECT
            customer_state AS state, 
            count(customer_id) AS customers,
            CASE 
            WHEN customer_state IN ('AC','AP','AM','PA','RO','RR','TO') THEN 'Norte'
            WHEN customer_state IN ('AL','BA','CE','MA','PB','PE','PI','RN','SE') THEN 'Nordeste'
            WHEN customer_state IN ('ES','MG','RJ','SP') THEN 'Sudeste'
            WHEN customer_state IN ('DF','GO','MT','MS') THEN 'Centro Oeste'
            WHEN customer_state IN ('PR','RS','SC') THEN 'Sul'
            END AS region

        FROM 
            `dbt_shop.customers` AS customers
        GROUP BY 1
        ORDER BY 1) AS region
    GROUP BY region
```

Comando utilizado via terminal no dbt

``dbt run --models shop.customers_by_region``


Prints

![Comando no Terminal](atividade3/image-10.png)
![Comando no Terminal](atividade3/image-11.png)
![Comando no Terminal](atividade3/image-12.png)

5. Crie uma tabela `items_costs` que contenha o preço total de um item (soma do valor do produto com o frete).

A query SQL

items_costs.sql
   
   ```sql
{{ config(materialized='table') }}

      SELECT *,
            price + freight_value as total_price
      FROM `dbt_shop.items`  
```

Comando utilizado via terminal no dbt

``dbt run --models shop.items_costs``


Prints

![Comando no Terminal](atividade3/image-13.png)
![Comando no Terminal](atividade3/image-14.png)
![Comando no Terminal](atividade3/image-15.png)

6. Crie uma `view` com o nome `orders_shipped_costs`, você deve calcular o preço total do pedido somado com o frete, apenas para pedidos enviado (shipped).

A query SQL

orders_shipped_costs.sql
   
   ```sql
{{ config(materialized='view') }}

WITH frete AS
  (SELECT DISTINCT
      orders.order_id as order_id,
      items.freight_value as frete 
  FROM 
      `dbt_shop.orders` AS orders
 INNER JOIN `dbt_shop.items` AS items
    ON orders.order_id = items.order_id
 GROUP BY 1,2
 ORDER BY 1)

 SELECT
     orders.order_id,
     sum(items.price) + sum(frete.frete) as total_price
 FROM 
     `dbt_shop.orders` AS orders
 INNER JOIN `dbt_shop.items` AS items
 ON orders.order_id = items.order_id
 INNER JOIN frete as frete
 ON orders.order_id = frete.order_id
 WHERE orders.order_status = 'shipped'
 GROUP BY 1
 ORDER BY 1
```

Comando utilizado via terminal no dbt

``dbt run --models shop.orders_shipped_costs``



Prints

![Comando no Terminal](atividade3/image-16.png)
![Comando no Terminal](atividade3/image-17.png)
![Comando no Terminal](atividade3/image-18.png)

